In [9]:
import os

In [10]:
%pwd

'd:\\software\\python_vs\\lone_approval_prediction'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\software\\python_vs\\lone_approval_prediction'

In [16]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    

In [17]:
import os
import zipfile
import urllib.request as request
from lone_approval_prediction.constants import *
from lone_approval_prediction import logger
from lone_approval_prediction.utils.comman import create_directories, read_yaml

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config["artifacts_root"]])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config["data_ingestion"]
        
        create_directories([config["root_dir"]])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config["root_dir"],
            source_url = config["source_url"],
            local_data_file = config["local_data_file"],
            unzip_dir = config["unzip_dir"]
            
        )
        
        return data_ingestion_config
    

In [18]:
import zipfile
import pandas
import shutil
import os
import urllib.request as request
from pathlib import Path
from lone_approval_prediction import logging
from lone_approval_prediction.utils.comman import get_size

class DataIngestion:
    def __init__(self, config):
        self.config = config

    def download_file(self):
        local_data_file_dir = os.path.dirname(self.config.local_data_file)
        if not os.path.exists(local_data_file_dir):
            os.makedirs(local_data_file_dir, exist_ok=True)

        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded successfully with following info:\n{headers}")
        else:
            logger.info(f"file already exists of size {get_size(self.config.local_data_file)}")

    def extract_zipfile(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Extracted all files at {unzip_path}")


In [20]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()
except Exception as e:
    raise e

2025-01-30 22:10:26,194: INFO: comman: yaml file : config\config.yaml lodded successfully
2025-01-30 22:10:26,197: INFO: comman: yaml file : params.yaml lodded successfully
2025-01-30 22:10:26,202: INFO: comman: yaml file : schema.yaml lodded successfully
2025-01-30 22:10:26,202: INFO: comman: created directory at :artifacts
2025-01-30 22:10:26,207: INFO: comman: created directory at :artifacts/data_ingestion


EnsureError: Argument path of type <class 'str'> to <function get_size at 0x000001CAD1802D40> does not match annotation type <class 'pathlib.Path'>